In [ ]:
import splitfolders


# In[19]:


input_folder = r'D:\dataset\archive'


# In[21]:


splitfolders.ratio(input_folder, output=r'D:\dataset\archive\split', 
                   seed=42, ratio=(.8, .1,.1), 
                   group_prefix=None) # default values

In [1]:
import torch
import torchvision
import torchvision.transforms as T
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import cv2

In [2]:
data_dir = r"D:\dataset\archive\split"
transform = T.Compose([
            T.RandomHorizontalFlip(),
            T.RandomVerticalFlip(),
            T.RandomApply(torch.nn.ModuleList([T.ColorJitter()]), p=0.25),
            T.Resize(224),
            T.CenterCrop(224),
            T.ToTensor(),
            T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)), # imagenet means
            T.RandomErasing(p=0.2, value='random')
        ])
train_data = torchvision.datasets.ImageFolder(os.path.join(data_dir, "train"), transform = transform)
train_video_names = [os.path.splitext(os.path.basename(sample[0]))[0] for sample in train_data.samples]
print(train_video_names)

metadataPath= r"D:\dataset\archive\metadata.csv"
metadata_df = pd.read_csv(metadataPath)
video_names = metadata_df.videoname.tolist()
video_names = [name.replace('.mp4', '') for name in video_names]

labels = metadata_df.label.to_list()
labels = [ 0 if label == "FAKE" else 1 for label in labels]


paired_data = []
for i in range(len(train_data)):
    video_name = train_video_names[i]
    label_index = video_names.index(video_name)
    paired_data.append((train_data[i][0], labels[label_index]))

train_loader = torch.utils.data.DataLoader(paired_data, batch_size=16, shuffle=True, num_workers=1)


['aaagqkcdis', 'aaavbjopeq', 'aaavflklag', 'aabdogagch', 'aabgnwnqba', 'aabjfjbpir', 'aabkwhhgwv', 'aabmgfhcgg', 'aabqcugcpc', 'aabuyfvwrh', 'aabyjxwqyo', 'aacehwwzol', 'aacfuwiwjw', 'aacmrrcpmr', 'aacweweyzz', 'aacwokkodz', 'aadlvezwah', 'aadqbokerz', 'aadubxfxhr', 'aaeflzzhvy', 'aaerwkkcfy', 'aaeucwtkdx', 'aaewstdhzp', 'aafcgzwvmy', 'aafezqchru', 'aafkeenkqm', 'aafuphyhpt', 'aaghqpewzx', 'aagkcljeci', 'aagnpclvlz', 'aagondskwl', 'aagpxhuqpp', 'aagsmskgyw', 'aagumdmtjk', 'aagundkpoh', 'aahgdqgozs', 'aahkgsltxw', 'aahncigwte', 'aahntrlswm', 'aahslbfkop', 'aahsnkchkz', 'aaiqsitvnd', 'aaivbtdyjn', 'aajhbrxhzm', 'aajtsolpjq', 'aajxdztmpb', 'aakbpusmxs', 'aakdoghgcp', 'aakjdfqwnu', 'aakjukgjme', 'aakkdgsmvl', 'aakrdapxha', 'aakxolbwgg', 'aakzzdrumy', 'aalkxecnzm', 'aalolkuhxb', 'aamjfukxwp', 'aamrbolklk', 'aannvfgmek', 'aanvfngndg', 'aaoecyszps', 'aaonlmxhne', 'aaooydetsr', 'aaoyvnkeyo', 'aapauvxnsf', 'aapjptyetv', 'aapqbyzjws', 'aaprcuxsdb', 'aaqdkpivxv', 'aaqnqubunq', 'aarrrzwibf', 'aars

In [ ]:
 # val/test
testTransform = T.Compose([ # We dont need augmentation for test transforms
            T.Resize(224),
            T.CenterCrop(224),
            T.ToTensor(),
            T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)), # imagenet means
        ])
val_data = torchvision.datasets.ImageFolder(os.path.join(data_dir, "val/"), transform=testTransform)
test_data = torchvision.datasets.ImageFolder(os.path.join(data_dir, "test/"), transform=testTransform)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=32, shuffle=True, num_workers=1)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=True, num_workers=1)

In [ ]:
"""
    Propsoed CNN architecture.
    
"""

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        # Pamameters Initialization
        input_shape = (3,224,224)
        activation = nn.ReLU()
        padding = 1
        droprate = 0.1
        epsilon=0.001

        self.layer1 = nn.Sequential(
            nn.BatchNorm2d(num_features=input_shape[0]),
            nn.Conv2d(in_channels=input_shape[0], out_channels=16, kernel_size=3, stride=1, padding=padding),
            activation,
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.BatchNorm2d(num_features=16, eps=epsilon)
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=padding),
            activation,
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.BatchNorm2d(num_features=32, eps=epsilon),
            nn.Dropout2d(p=droprate)
        )

        self.layer3 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=padding),
            activation,
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.BatchNorm2d(num_features=64, eps=epsilon),
            nn.Dropout2d(p=droprate)
        )

        self.layer4 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=padding),
            activation,
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.BatchNorm2d(num_features=128, eps=epsilon),
            nn.Dropout2d(p=droprate)
        )

        self.layer5 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=padding),
            activation,
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.BatchNorm2d(num_features=256, eps=epsilon),
            nn.Dropout2d(p=droprate)
        )

        self.layer6 = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=padding),
            activation,
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.BatchNorm2d(num_features=512, eps=epsilon),
            nn.Dropout2d(p=droprate)
        )

        self.layer7 = nn.Sequential(
            nn.AdaptiveAvgPool2d(output_size=(1, 1)),
            nn.Flatten(),
            nn.Linear(512, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.layer6(x)
        x = self.layer7(x)
        return x

model = CNN()

In [ ]:
#Training phase imp:

device = torch.device("cuda")
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        print("input is ", data[0], "and label is ", data[1])
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 100 == 99:    
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0
